In [0]:
try:
  import googlemaps
  
  # For decoding Google's directions API polyline
  import polyline 
  from shapely.geometry import Point, LineString
  
  googlemaps_module = True
except ImportError:
  googlemaps_module = False

In [0]:
import socket
import time
import numpy as np
import re
from datetime import datetime
import pandas as pd
socket.setdefaulttimeout(120)

In [0]:
import googlemaps
import pandas as pd
import numpy as np
import time

class GoogleMaps:
    """A class to encapsulate Google Maps geocoding for retrieving latitude and longitude based on flexible address combinations."""

    def __init__(self):
        """Initialize a Google Maps server connection

        ARGUMENTS:
        secret_scope : str -- Databricks secret scope to get authentication details
        """
        google_api_key = ''
        self.gmaps = googlemaps.Client(key=google_api_key)

    def get_coordinates(self, row, address_fields):
        """Constructs an address from row fields and retrieves coordinates.

        ARGUMENTS:
        row           : pd.Series -- Row of a DataFrame containing address fields.
        address_fields: list -- List of column names to combine for address (e.g., ['street', 'barangay', 'city', 'province']).

        RETURNS:
        pd.Series containing latitude and longitude if found, otherwise None.
        """
        address_combinations = [
            ", ".join([str(row[field]) for field in address_fields if pd.notnull(row[field])])
        ]

        for address in address_combinations:
            if address.strip():  # Ensure address is not empty
                try:
                    request = self.gmaps.geocode(address)
                    if request and 'geometry' in request[0]:
                        location = request[0]['geometry']['location']
                        return pd.Series([location['lat'], location['lng']])
                except Exception as e:
                    print(f"Error fetching coordinates for address '{address}': {e}")
                    continue
        
        # If no valid coordinates found, return None
        return pd.Series([None, None])

    def add_coordinates(self, df, address_fields):
        """Adds Latitude and Longitude columns to the DataFrame based on address fields.

        ARGUMENTS:
        df            : pd.DataFrame -- DataFrame with address fields to geocode.
        address_fields: list -- List of column names to combine for address (e.g., ['street', 'barangay', 'city', 'province']).

        RETURNS:
        pd.DataFrame with added 'Latitude' and 'Longitude' columns.
        """
        # Apply get_coordinates function to each row of DataFrame and add Latitude and Longitude
        df[['Latitude', 'Longitude']] = df.apply(self.get_coordinates, axis=1, address_fields=address_fields)
        return df


In [0]:
# Example usage:
gmaps = GoogleMaps()
address_fields = ['street', 'barangay', 'city', 'province']  # Adjust based on your DataFrame
df = pd.read_excel("your_file.xlsx")

# Add coordinates
df_with_coords = gmaps.get_coordinates(df, address_fields)

# Save the updated DataFrame to a new Excel file
df_with_coords.to_excel("updated_with_coordinates.xlsx", index=False)


In [0]:
gmaps = googlemaps.Client(key='')

In [0]:
gmaps.geocode('D SAMONTE STREET, SAN LORENZO (POBLACION), LAOAG CITY, ILOCOS NORTE', 'textquery')